In [1]:
# imports
import os,sys
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import time

# import some helper functions
from proc_helpers import buffer_point_polygon_overlay,pp_summarize_ndvi_with_qa_dir

In [2]:
points_shp = '../../gis/moved_locations.shp'
extra_erase_shp = ['../../gis/VillplusGR.shp', '../../gis/Agin_Villages.shp']

points_shp = '../locs_for_JM//moved_locations.shp'
extra_erase_shp = ['../testing_villages_new/VillplusGR.shp', '../testing_villages_new/Agin_Villages.shp']

df = gpd.read_file(points_shp)
print('point shapefile crs: {}'.format(df.crs))
print('erase file 1 shapefile crs: {}'.format(gpd.read_file(extra_erase_shp[0]).crs))
print('erase file 2 shapefile crs: {}'.format(gpd.read_file(extra_erase_shp[1]).crs))

# ensure the points are in the same CRS as polygons... for some reason they are different
df = df.to_crs(epsg='32736')

point shapefile crs: {'init': 'epsg:32636'}
erase file 1 shapefile crs: {'init': 'epsg:32736'}
erase file 2 shapefile crs: {'init': 'epsg:32736'}


In [3]:
df.crs == {'init':'epsg:32734'}
df.head()

,VillageNam,LocationID,NewName,NewID,geometry
0,Agincourt,BBBCE,Agincourt,01,POINT (323325.9884000003 7254571.9086)
1,Agincourt,BBBCN,Agincourt,01,POINT (323793.302 7254205.93)
2,Agincourt,BBBEH,Agincourt,01,POINT (323905.9172000003 7253881.413899999)
3,Agincourt,BBBEN,Agincourt,01,POINT (323416.6338000002 7253991.3725)
4,Agincourt,BBBJY,Agincourt,01,POINT (324492.3450999995 7252748.390000001)


In [4]:
# buff_erase_df = create_buffer_point_polygon_overlay(df, method='difference', num_points=2)

# use this one for the points specified in the shapefile (_v2)
# buff_erase_df = create_buffer_point_polygon_overlay_v2(df, method='difference') 
# buff_erase_df.crs = {'init' :'epsg:32736'}

# use this one to erase a list of shapefiles from the buffered points. Those shapefiles must have same CRS
#buff_erase_df = create_buffer_point_polygon_overlay_v3(df, method='difference', num_points_fld='NUMHH', erase_shp_files=[extra_erase_shp]) 

# use this one if providing a data frame of points and to erase a list of shapefiles from buffered points.
# test_df = df[df['NewID'].astype('int') < 3]
# test_df = df.iloc[100:200]
buff_erase_df = buffer_point_polygon_overlay(df, erase_shp_files=extra_erase_shp)

#buff_erase_df.plot()

{'init': 'epsg:32736'} {'init': 'epsg:32736', 'no_defs': True}
{'init': 'epsg:32736'} {'init': 'epsg:32736', 'no_defs': True}


## get the NDVI and pixel_qa raster paths and check that they match up
### specify the qa_dir and ndvi_dir with trailing '/' character or the functions will not run

In [5]:
qa_dir = '../landsat/test/qa/'
ndvi_dir = '../landsat/test/ndvi/'


In [6]:
t0 = time.time()
test_function_pp = pp_summarize_ndvi_with_qa_dir(ndvi_dir, qa_dir, buff_erase_df, 'median')
t1 = time.time()

## the number of cores and geometries will change...
#print('parallel across 6 cores and 28 geometries is: {}'.format(t1-t0))


## don't run this, it operates in serial and is slow
# t0 = time.time()
# test_function = summarize_ndvi_with_qa_dir(ndvi_dir, qa_dir, buff_erase_df, 'median')
# t1 = time.time()
# print('serial and 28 geometries is: {}'.format(t1-t0))

epsg_df: 32736, epsg_raster: 32636
epsg_df: {'init': 'epsg:32636', 'no_defs': True}, epsg_raster: 32636


C:\projects\RD\south_africa\jp\proc_helpers.py:583: UserWarning: Warning: converting a masked element to nan.
  ndvi_df = geo_df.join(pd.DataFrame(np.array(all_vals).T, columns=landsat_columns))


In [9]:
test_function_pp.shape, buff_erase_df.shape
test_function_pp.to_crs(epsg=32736).geometry

0        (POLYGON ((325325.9884000005 7254571.9086, 325...
1        (POLYGON ((325793.3019999997 7254205.93, 32578...
2        (POLYGON ((325905.9172000005 7253881.413900002...
3        (POLYGON ((325416.6338 7253991.3725, 325412.38...
4        (POLYGON ((326467.9020947113 7252452.498234466...
5        (POLYGON ((325815.1298000001 7252494.410399999...
6        (POLYGON ((325686.6436999999 7251898.2533, 325...
7        (POLYGON ((325458.6084999998 7252114.8038, 325...
8        (POLYGON ((325777.8543999993 7251670.320600001...
9        POLYGON ((324983.2472999999 7251062.143300001,...
10       (POLYGON ((325382.943 7253092.604500001, 32537...
11       (POLYGON ((325822.1676000003 7252815.132200001...
12       (POLYGON ((325426.6230560013 7253579.194278575...
13       (POLYGON ((324909.2825728823 7253634.541708498...
14       (POLYGON ((325027.8514 7253435.2365, 325018.22...
15       (POLYGON ((324879.8822 7253222.739399999, 3248...
16       (POLYGON ((319860.8133982481 7253807.226716048.

In [ ]:
# change the output filename to your desire
test_function_pp.to_crs(epsg=32736).to_file('test_landsat_summaries_1997_2017_median_maskCloudWater.shp')

In [20]:
all_vals_arr
landsat_columns
ndvi_df.shape, buff_erase_df.shape, buff_erase_df.join(pd.DataFrame(all_vals_arr.T, columns=landsat_columns)).shape
junk = buff_erase_df.join(pd.DataFrame(all_vals_arr.T, columns=landsat_columns))
junk.to_crs(epsg=32636).geometry

100    (POLYGON ((317954.3460191695 -2746243.33964341...
101    (POLYGON ((321356.8258999994 -2747425.1595, 32...
102    (POLYGON ((316767.9601816834 -2746526.32509758...
103    (POLYGON ((321230.9337000001 -2746526.5154, 32...
104    (POLYGON ((325665.8746999993 -2747785.6074, 32...
105    (POLYGON ((325777.6059000003 -2747934.8996, 32...
106    POLYGON ((325244.4335999999 -2748905.152499999...
107    POLYGON ((324904.9069999998 -2748775.6525, 324...
108    (POLYGON ((325081.5504999995 -2746533.63169999...
109    (POLYGON ((324228.4599686214 -2746029.84972034...
110    (POLYGON ((324945.2107999996 -2746833.67549999...
111    (POLYGON ((324206.5346480416 -2747243.20823820...
112    (POLYGON ((323497.4941881252 -2748256.54339274...
113    (POLYGON ((322380.1996000001 -2747222.58000000...
114    (POLYGON ((318795.9064999663 -2746375.26887053...
115    (POLYGON ((322046.5409865034 -2747347.90230201...
116    (POLYGON ((318343.1015772807 -2746215.96042852...
117    (POLYGON ((317832.857252

In [20]:
## something happens with the to_crs call. not sure why the CRS issue exists, but it does! so let's try something else
## like... just assigning the crs.
from fiona.crs import from_epsg
landsat_crs = from_epsg(32636)

In [21]:
test_crs_buff = buff_erase_df.copy()
test_crs_buff.crs = landsat_crs
t0 = time.time()
test_crs = pp_summarize_ndvi_with_qa_dir(ndvi_dir, qa_dir, test_crs_buff, 'median')
t1 = time.time()

epsg_df: 32636, epsg_raster: 32636
epsg_df: {'init': 'epsg:32636', 'no_defs': True}, epsg_raster: 32636


ValueError: Input shapes do not overlap raster.

# everything below here is testing and doesn't need to run
## test number of points for scaling

In [25]:
buff_erase_df = buffer_point_polygon_overlay(df, erase_shp_files=extra_erase_shp)
test_b = buff_erase_df.iloc[100:200]
test_g = buff_erase_df.iloc[:100]

{'init': 'epsg:32736'} {'init': 'epsg:32736', 'no_defs': True}
{'init': 'epsg:32736'} {'init': 'epsg:32736', 'no_defs': True}


In [21]:
test_g.crs, test_b.crs

({'init': 'epsg:32736', 'no_defs': True},
 {'init': 'epsg:32736', 'no_defs': True})

In [34]:
test_b.crs

{'init': 'epsg:32736', 'no_defs': True}